# AD9833 ToolBox test with MicroPython Remote
- https://www.analog.com/media/en/technical-documentation/data-sheets/ad9833.pdf
- https://pypi.org/project/jupyter-micropython-remote/
- https://towardsdatascience.com/micropython-on-esp-using-jupyter-6f366ff5ed9

In [ ]:
%local
!ls /dev/ttyUSB*

In [ ]:
# %connect /dev/ttyUSB0 --baudrate=115200 --user='micro' --password='python' --wait=0

%connect COM4 --baudrate=115200 --user='micro' --password='python' --wait=0

In [ ]:
# %lsmagic

In [ ]:
import peripherals
import tools
from ad9833 import AD9833

import machine
import gc

gc.collect()

## Debug mode?

In [ ]:
AD9833.DEBUG_MODE_SHOW_BUS_DATA = False        # whether to show bus data. 
AD9833.DEBUG_MODE_PRINT_REGISTER = False        # whether to print registers. 

## Generators

In [ ]:
with_hardware_device = True

if with_hardware_device:
    _spi = peripherals.SPI.get_uPy_spi(polarity = 1)
    _ss1 = peripherals.Pin.get_uPy_pin(pin_id = 15, output = True) 
    _ss2 = peripherals.Pin.get_uPy_pin(pin_id =  4, output = True) 
else:
    _spi = _ss1 = _ss2 = None  # using None for testing without actual hardware device.

ad1 = AD9833(_spi, _ss1)
ad2 = AD9833(_spi, _ss2)

In [ ]:
ad1.reset()
ad2.reset()

In [ ]:
ad1.enable_output(False)
ad2.enable_output(False)

## Frequency Sweeping

In [ ]:
tb = tools.ToolBox()

In [ ]:
tb.sweep(ad1, freq_start = 10, freq_end = int(1e6), n_freqs = 100,
         sweep_type = 'logarithm', direction = 'round_trip', n_cycles = 1,
         slot_duration = 0.01, between_cycle_seconds = 0);

## Function Toggling

In [ ]:
duration_seconds, count, cycle_time = tb.toggle(ad1,
                                                fun = 'enable_output', params = ({'value': True}, {'value': False}),
                                                n_cycles = 3, slot_duration = 0.2, between_cycle_seconds = 0.2)
print('duration_seconds, count, cycle_time', duration_seconds, count, cycle_time)

## Waveform Juggling

In [ ]:
tb.juggle((ad1, ad2),
          freq_start = int(1e2), freq_end = int(2e4), n_freqs = 100, freqs_type = 'logarithm',
          slot_duration = 0.2, between_cycle_seconds = 0.2,
          n_juggles = 10)